In [1]:
from google.colab import files
uploaded = files.upload()


Saving tab_aml_polars_version_thirtyeight.py to tab_aml_polars_version_thirtyeight.py


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
DATASET_DIR = "/content/drive/MyDrive/kaggle_datasets/SAML-D"

In [4]:
import polars as pl


In [5]:
df = pl.read_csv(f"{DATASET_DIR}/SAML-D.csv")
df.head()

Time,Date,Sender_account,Receiver_account,Amount,Payment_currency,Received_currency,Sender_bank_location,Receiver_bank_location,Payment_type,Is_laundering,Laundering_type
str,str,i64,i64,f64,str,str,str,str,str,i64,str
"""10:35:19""","""2022-10-07""",8724731955,2769355426,1459.15,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,"""Normal_Cash_Deposits"""
"""10:35:20""","""2022-10-07""",1491989064,8401255335,6019.64,"""UK pounds""","""Dirham""","""UK""","""UAE""","""Cross-border""",0,"""Normal_Fan_Out"""
"""10:35:20""","""2022-10-07""",287305149,4404767002,14328.44,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cheque""",0,"""Normal_Small_Fan_Out"""
"""10:35:21""","""2022-10-07""",5376652437,9600420220,11895.0,"""UK pounds""","""UK pounds""","""UK""","""UK""","""ACH""",0,"""Normal_Fan_In"""
"""10:35:21""","""2022-10-07""",9614186178,3803336972,115.25,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,"""Normal_Cash_Deposits"""


In [6]:
import torch
from tab_aml_polars_version_thirtyeight import preprocess_saml_d, FocalLoss, fast_hash, recast, collate_encoded_batch, custom_split_polars, TabAMLDataset, TabAMLModel, train_epoch, evaluate, compute_pos_weight, build_weighted_sampler
from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
import torch.nn as nn

In [7]:
df = preprocess_saml_d(df)

In [8]:
df = recast(df)

In [9]:
df.head(5)

Date,Sender_account,Receiver_account,Payment_currency,Received_currency,Sender_bank_location,Receiver_bank_location,Payment_type,Is_laundering,timestamp,day,month,year,hour,day_of_week,is_weekend,amount_log
date,i64,i64,str,str,str,str,str,i8,datetime[μs],i8,i8,i16,i8,i8,i8,f64
2022-10-07,8724731955,2769355426,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,2022-10-07 10:35:19,7,10,2022,10,5,1,7.286294
2022-10-07,1491989064,8401255335,"""UK pounds""","""Dirham""","""UK""","""UAE""","""Cross-border""",0,2022-10-07 10:35:20,7,10,2022,10,5,1,8.702949
2022-10-07,287305149,4404767002,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cheque""",0,2022-10-07 10:35:20,7,10,2022,10,5,1,9.570071
2022-10-07,5376652437,9600420220,"""UK pounds""","""UK pounds""","""UK""","""UK""","""ACH""",0,2022-10-07 10:35:21,7,10,2022,10,5,1,9.383957
2022-10-07,9614186178,3803336972,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,2022-10-07 10:35:21,7,10,2022,10,5,1,4.755743


In [10]:
train_df, val_df, test_df = custom_split_polars(df, validation_days=70, test_days=35)



Split complete:
  Train: 6397772
  Val: 2076692
  Test: 1030388


In [11]:
print(len(train_df), len(val_df), len(test_df))

6397772 2076692 1030388


In [12]:
cat_cols = ["Sender_account", "Receiver_account", "Payment_currency", "Received_currency", "Sender_bank_location",
            "Receiver_bank_location", "Payment_type", "day", "month", "year", "hour", "day_of_week", "is_weekend"]

In [13]:
cont_cols = ["amount_log"]

In [ ]:
train_ds = TabAMLDataset(train_df, cat_cols, cont_cols, "Is_laundering", fit=True)

In [ ]:
val_ds   = TabAMLDataset(val_df, cat_cols, cont_cols, "Is_laundering", encoders=train_ds.encoders, scaler=train_ds.scaler, fit=False)

In [ ]:
test_ds  = TabAMLDataset(test_df, cat_cols, cont_cols, "Is_laundering", encoders=train_ds.encoders, scaler=train_ds.scaler, fit=False)


In [ ]:
train_sampler = build_weighted_sampler(pl.Series(train_df["Is_laundering"]))

In [ ]:
train_loader  = DataLoader(train_ds, batch_size=512, sampler=train_sampler, collate_fn=collate_encoded_batch)


In [ ]:
val_loader   = DataLoader(val_ds, batch_size=512, shuffle=False, collate_fn=collate_encoded_batch)

In [ ]:
test_loader  = DataLoader(test_ds, batch_size=512, shuffle=False, collate_fn=collate_encoded_batch)

In [ ]:
model = TabAMLModel(category_sizes=train_ds.category_sizes, cont_dim=len(cont_cols), embedding_dim=64, shared_ratio=1/6, num_heads=4, num_layers1=2, num_layers2=2, dropout=0.25, micro_indices=(0, 1))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
loss_fn = FocalLoss(alpha=0.75, gamma=2.0)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=0.5, patience=2, verbose=True, min_lr=1e-6
)

In [ ]:
epochs = 50
best_val_pr = 0
patience, bad_epochs = 5, 0

for epoch in range(1, epochs + 1):
    train_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
    val_loss, val_auc, val_pr, val_rec, val_prec, val_f1, _ = evaluate(model, val_loader, loss_fn, device)

    scheduler.step(val_pr)

    print(f"Epoch {epoch}/{epochs} | Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | ROC-AUC: {val_auc:.4f} | "
          f"PR-AUC: {val_pr:.4f} | Recall: {val_rec:.3f} | Precision: {val_prec:.3f} | F1: {val_f1:.3f}")

    if val_pr > best_val_pr:
        best_val_pr = val_pr
        torch.save(model.state_dict(), "best_tabaml_model.pt")
        bad_epochs = 0
    else:
        bad_epochs += 1
        if bad_epochs >= patience:
            print("Early stopping triggered.")
            break


In [ ]:
model.load_state_dict(torch.load("best_tabaml_model.pt"))

best_thr, best_prec, best_rec = pick_threshold_by(val_loader, model, device, criterion="f1")
print(f"Best threshold: {best_thr:.3f} | Precision: {best_prec:.3f} | Recall: {best_rec:.3f}")


In [ ]:
test_loss, test_auc, test_pr, test_rec, test_prec, test_f1, cm = evaluate(
    model, test_loader, loss_fn, device, threshold=best_thr
)

print("\n FINAL TEST RESULTS")
print(f"Loss: {test_loss:.4f} | ROC-AUC: {test_auc:.4f} | PR-AUC: {test_pr:.4f}")
print(f"Recall: {test_rec:.3f} | Precision: {test_prec:.3f} | F1: {test_f1:.3f}")
print(f"Confusion Matrix: {cm}")


In [ ]:
from sklearn.metrics import PrecisionRecallDisplay
import matplotlib.pyplot as plt

y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for batch in test_loader:
        logits = model(batch.x_cat.to(device), batch.x_cont.to(device))
        y_true.extend(batch.y.numpy())
        y_pred.extend(torch.sigmoid(logits).cpu().numpy())

disp = PrecisionRecallDisplay.from_predictions(y_true, y_pred)
disp.ax_.set_title("Precision-Recall Curve (Test Set)")
plt.show()
